Milestone 2
COVID-19 and American Community Survey Data Exploration

Create new BQ dataset and remodel

In [2]:
dataset_id = "JHU_refined" # Covid-19 data
!bq --location=US mk --dataset {dataset_id}

dataset_id = "Census_refined" # Census data
!bq --location=US mk --dataset {dataset_id}

Dataset 'lone-ranger:JHU_refined' successfully created.
Dataset 'lone-ranger:Census_refined' successfully created.


FIX COVID JHU SCHEMA

In [199]:
%%bigquery
create or replace table JHU_refined.CovidLocation as
select distinct null as id, Province_State, Country_Region
from JHU_staging.Nov2

""


In [200]:
%%bigquery
update JHU_refined.CovidLocation set id =FARM_FINGERPRINT(Province_State)
where Country_Region is not null

""


In [201]:
%%bigquery
select count(*)
from JHU_refined.CovidLocation
limit 5

,f0_
0,58


In [202]:
%%bigquery
create or replace table Census_refined.CensusLocation as
select GeoID, GeoName from Census_staging.SeniorGen
union Distinct (select GeoID, GeoName from Census_staging.PovertyGen)
union Distinct (select GeoID, GeoName from Census_staging.Insurance)
union Distinct (select GeoID, GeoName from Census_staging.SeniorGen)

""


In [203]:
%%bigquery
select * from Census_refined.CensusLocation
order by GeoName
limit 5

,GeoID,GeoName
0,97000US0601620,"ABC Unified School District, California"
1,31000US10140,"Aberdeen, WA Micro Area"
2,40000US00199,"Aberdeen--Bel Air South--Bel Air North, MD Urb..."
3,97000US4807440,"Abilene Independent School District, Texas"
4,16000US4801000,"Abilene city, Texas"


In [204]:
%%bigquery
select count(*) as null_count
from Census_refined.CensusLocation
where GeoID is null

,null_count
0,0


In [205]:
%%bigquery
create or replace table JHU_refined.CovidLocation as
select id, Province_State, Country_Region
from JHU_refined.CovidLocation as l1
join Census_refined.CensusLocation as l2
on l1.Province_State = l2.GeoName

""


In [206]:
%%bigquery
select count(*)
from JHU_refined.CovidLocation

,f0_
0,52


In [207]:
%%bigquery
select *
from JHU_refined.CovidLocation
limit 5

,id,Province_State,Country_Region
0,-6327485967938119003,Alaska,US
1,8853016079165984657,Tennessee,US
2,5584253583079874476,New Hampshire,US
3,6998689060605007651,Kansas,US
4,-7292501111997658044,Ohio,US


In [208]:
%%bigquery
create or replace table Census_refined.CensusLocation as
select GeoID as Geo_ID, GeoName
from JHU_refined.CovidLocation as l1
join Census_refined.CensusLocation as l2
on l1.Province_State = l2.GeoName

""


In [209]:
%%bigquery
select count(*)
from Census_refined.CensusLocation

,f0_
0,52


In [210]:
%%bigquery
select *
from Census_refined.CensusLocation
limit 5

,Geo_ID,GeoName
0,04000US20,Kansas
1,04000US28,Mississippi
2,04000US08,Colorado
3,04000US02,Alaska
4,04000US26,Michigan


In [211]:
%%bigquery
create or replace table JHU_refined.LocLinker as
select id as Covid_ID, Geo_ID, Province_State
from JHU_refined.CovidLocation as l1
join Census_refined.CensusLocation as l2
on l1.Province_State = l2.GeoName

""


In [212]:
%%bigquery
select count(*)
from JHU_refined.LocLinker

,f0_
0,52


In [213]:
%%bigquery
select *
from JHU_refined.LocLinker
limit 5

,Covid_ID,Geo_ID,Province_State
0,6998689060605007651,04000US20,Kansas
1,3991597989243244703,04000US49,Utah
2,9215448666838151517,04000US12,Florida
3,99786220189605989,04000US37,North Carolina
4,-6327485967938119003,04000US02,Alaska


In [214]:
%%bigquery
create or replace table JHU_staging.USAug17Cases as
select (select id from JHU_refined.CovidLocation where CovidLocation.Province_State = USAug17Cases.Province_State) as l_id, Last_Update, Confirmed, Deaths, CAST(Recovered as int64) as Recovered, CAST(Active as int64) as Active, CAST(People_Tested as int64) as People_Tested, Mortality_Rate, Testing_Rate
from JHU_staging.USAug17Cases

""


In [215]:
%%bigquery
select * from JHU_staging.USAug17Cases
where Recovered != 0
limit 3

,l_id,Last_Update,Confirmed,Deaths,Recovered,Active,People_Tested,Mortality_Rate,Testing_Rate
0,6499411745341259851,2020-08-18 04:30:23+00:00,72412,2095,56577,13740,524589,2.893167,17626.436042
1,8871368517206463294,2020-08-18 04:30:23+00:00,107421,2385,13827,91209,1373442,2.220236,16090.902030
2,7577259256921371390,2020-08-18 04:30:23+00:00,81006,3135,60677,17194,914633,3.870084,13585.906816


In [217]:
%%bigquery
create or replace table JHU_staging.USFallCases as
select (select id from JHU_refined.CovidLocation where CovidLocation.Province_State = USFallCases.Province_State) as l_id, Last_Update, Confirmed, Deaths, CAST(Recovered as int64) as Recovered, CAST(Active as int64) as Active, CAST(People_Tested as int64) as People_Tested, Mortality_Rate, Testing_Rate
from JHU_staging.USFallCases

""


In [218]:
%%bigquery
select * from JHU_staging.USFallCases
where Recovered != 0
limit 3

,l_id,Last_Update,Confirmed,Deaths,Recovered,Active,People_Tested,Mortality_Rate,Testing_Rate
0,NaN,2020-10-13 04:30:36+00:00,77,2,29,46,15198,2.597403,27560.568693
1,NaN,2020-09-19 04:30:26+00:00,2074,31,1450,593,45653,1.494696,27798.379093
2,NaN,2020-10-23 04:30:40+00:00,4056,69,2471,1516,62287,1.701183,37926.919119


In [220]:
%%bigquery
create or replace table JHU_staging.USJune29Cases as
select (select id from JHU_refined.CovidLocation where CovidLocation.Province_State = USJune29Cases.Province_State) as l_id, Last_Update, Confirmed, Deaths, CAST(Recovered as int64) as Recovered, CAST(Active as int64) as Active, CAST(People_Tested as int64) as People_Tested, Mortality_Rate, Testing_Rate
from JHU_staging.USJune29Cases

""


In [221]:
%%bigquery
select * from JHU_staging.USJune29Cases
where Recovered != 0
limit 3

,l_id,Last_Update,Confirmed,Deaths,Recovered,Active,People_Tested,Mortality_Rate,Testing_Rate
0,-7233201808565450925,2020-06-30 04:34:02+00:00,11376,507,6665,4204,109931,4.456751,11289.285700
1,-6433276884676225209,2020-06-30 04:34:02+00:00,19042,269,13322,5451,175272,1.412667,9060.756573
2,-6327485967938119003,2020-06-30 04:34:02+00:00,901,14,525,362,108709,1.553829,14860.193153


In [222]:
%%bigquery
create or replace table JHU_staging.USMidAugCases as
select (select id from JHU_refined.CovidLocation where CovidLocation.Province_State = USMidAugCases.Province_State) as l_id, Last_Update, Confirmed, Deaths, CAST(Recovered as int64) as Recovered, CAST(Active as int64) as Active, CAST(People_Tested as int64) as People_Tested, Mortality_Rate, Testing_Rate
from JHU_staging.USMidAugCases

""


In [223]:
%%bigquery
select * from JHU_staging.USMidAugCases
where Recovered != 0
limit 3

,l_id,Last_Update,Confirmed,Deaths,Recovered,Active,People_Tested,Mortality_Rate,Testing_Rate
0,NaN,2020-08-26 04:30:22+00:00,998,12,754,232,14611,1.202405,13621.023977
1,NaN,2020-08-25 04:30:29+00:00,984,11,712,261,14442,1.117886,13463.474662
2,NaN,2020-08-27 04:30:22+00:00,1030,14,801,215,14965,1.359223,13951.038520


In [224]:
%%bigquery
create or replace table JHU_staging.USSummerCases as
select (select id from JHU_refined.CovidLocation where CovidLocation.Province_State = USSummerCases.Province_State) as l_id, Last_Update, Confirmed, Deaths, CAST(Recovered as int64) as Recovered, CAST(Active as int64) as Active, CAST(People_Tested as int64) as People_Tested, Mortality_Rate, Testing_Rate
from JHU_staging.USSummerCases

""


In [225]:
%%bigquery
select * from JHU_staging.USSummerCases
where Recovered != 0
limit 3

,l_id,Last_Update,Confirmed,Deaths,Recovered,Active,People_Tested,Mortality_Rate,Testing_Rate
0,NaN,2020-05-08 02:32:36+00:00,66,4,54,8,1056,6.060606,984.450162
1,NaN,2020-08-02 04:35:05+00:00,421,8,341,72,9289,1.900238,8659.618898
2,NaN,2020-08-04 04:42:12+00:00,439,8,354,77,9436,1.822323,8796.658836


In [226]:
%%bigquery
select * from JHU_staging.USSummerCases
where Last_Update is null
order by Last_Update

,l_id,Last_Update,Confirmed,Deaths,Recovered,Active,People_Tested,Mortality_Rate,Testing_Rate
0,NaN,NaT,51,1,43.0,50.0,350.0,1.960784,326.285565
1,NaN,NaT,0,0,58545.0,-58545.0,NaN,NaN,NaN
2,NaN,NaT,0,0,47763.0,-47763.0,NaN,NaN,NaN
3,NaN,NaT,0,0,32988.0,-32988.0,NaN,NaN,NaN
4,NaN,NaT,0,0,NaN,NaN,3.0,NaN,5.391708
5,NaN,NaT,0,0,NaN,NaN,3.0,NaN,5.391708
6,NaN,NaT,0,0,NaN,NaN,3.0,NaN,5.391708
7,NaN,NaT,0,0,NaN,NaN,3.0,NaN,5.391708
8,NaN,NaT,0,0,52096.0,-52096.0,NaN,NaN,NaN
9,NaN,NaT,0,0,70337.0,-70337.0,NaN,NaN,NaN


In [227]:
%%bigquery
create or replace table JHU_staging.USSummerCases as
select * from JHU_staging.USSummerCases
where Last_Update is not null AND l_id is not null

""


In [230]:
%%bigquery
create or replace table JHU_refined.USCovidCases as
select * from JHU_staging.USAug17Cases
union distinct (select * from JHU_staging.USFallCases)
union distinct (select * from JHU_staging.USMidAugCases)
union distinct (select * from JHU_staging.USSummerCases)
union distinct (select * from JHU_staging.USJune29Cases)

""


In [231]:
%%bigquery
create or replace table JHU_refined.USCovidCases as
select null as id, * 
from JHU_refined.USCovidCases

""


In [232]:
%%bigquery
update JHU_refined.USCovidCases set id =FARM_FINGERPRINT(CONCAT(CAST(l_id AS STRING),CAST(Last_Update AS STRING)))
where Last_Update is not null

""


In [233]:
%%bigquery
select * from JHU_refined.USCovidCases
where l_id is not null
order by Last_Update DESC

,id,l_id,Last_Update,Confirmed,Deaths,Recovered,Active,People_Tested,Mortality_Rate,Testing_Rate
0,4883818791343824525,-9006277456342450938,2020-11-09 05:30:38+00:00,130984,2394,8562.0,120028,2214788,1.827704,38459.620306
1,7010676568565741269,4619292899676057352,2020-11-09 05:30:38+00:00,7693,152,5935.0,1606,676314,1.975822,50313.045859
2,7774965549498962857,-4727321918456285425,2020-11-09 05:30:38+00:00,171399,10149,131646.0,29604,6720526,5.921271,97504.868696
3,4953858045488694584,8139360781050291472,2020-11-09 05:30:38+00:00,229073,7945,128981.0,92147,5213961,3.468327,52208.227273
4,-4421342614149587015,8179363653179218705,2020-11-09 05:30:38+00:00,180862,2710,146311.0,31841,3077966,1.498380,54577.426329
...,...,...,...,...,...,...,...,...,...,...
10967,-4795243679005356723,-9129656388734033754,2020-04-12 23:18:15+00:00,1407,27,NaN,1380,14308,1.918977,888.421400
10968,-5374713606178619533,2113635095029469378,2020-04-12 23:18:15+00:00,12035,554,NaN,11481,41220,4.603241,1156.148159
10969,3857201026646812491,7006816170409177292,2020-04-12 23:18:15+00:00,3563,93,NaN,3470,21583,2.610160,460.300152
10970,7664424541782494035,-2243208293355838763,2020-04-12 23:18:15+00:00,20595,840,NaN,19755,104045,4.078660,2263.192950


FIX CENSUS SCHEMA

In [234]:
%%bigquery
create or replace table Census_refined.Overall as
select distinct HealthWork.GeoID as Geo_ID, 
HealthWork.Estimate as HealthWorker, 
Insurance.Estimate as Insurance, 
PovertyGen.Estimate as Poverty, 
PovSenior.Estimate as PovSenior,
SeniorGen.Estimate as SeniorGen
from Census_staging.HealthWork
join Census_staging.Insurance
on HealthWork.GeoID = Insurance.GeoID
join Census_staging.PovertyGen
on HealthWork.GeoID = PovertyGen.GeoID
join Census_staging.PovSenior
on HealthWork.GeoID = PovSenior.GeoID
join Census_staging.SeniorGen
on HealthWork.GeoID = SeniorGen.GeoID
join Census_refined.CensusLocation
on HealthWork.GeoID = CensusLocation.Geo_ID

""


In [235]:
%%bigquery
select * from Census_refined.Overall
order by Geo_ID
limit 5

,Geo_ID,HealthWorker,Insurance,Poverty,PovSenior,SeniorGen
0,04000US01,6.9,9.7,15.5,10.5,17.4
1,04000US02,5.7,12.2,10.1,6.9,12.4
2,04000US04,5.6,11.3,13.5,9.0,18.0
3,04000US05,6.8,9.1,16.2,10.5,17.4
4,04000US06,5.2,7.7,11.8,10.5,14.8


In [236]:
%%bigquery
select count(*) from Census_refined.Overall

,f0_
0,52


In [256]:
%%bigquery
create or replace table JHU_refined.USCovidCases as
select * 
from JHU_refined.USCovidCases
where l_id is not null

""


Queries

select all places where the percent of poverty is greater than the overall avg poverty

In [237]:
%%bigquery
select Overall.Geo_ID, CensusLocation.GeoName, Poverty, PovSenior, SeniorGen
from Census_refined.Overall
join Census_refined.CensusLocation on CensusLocation.Geo_ID = Overall.Geo_ID
where Poverty >= (select avg(Poverty) as avg_pov from Census_refined.Overall)
order by Geo_ID
limit 5

,Geo_ID,GeoName,Poverty,PovSenior,SeniorGen
0,04000US01,Alabama,15.5,10.5,17.4
1,04000US04,Arizona,13.5,9.0,18.0
2,04000US05,Arkansas,16.2,10.5,17.4
3,04000US11,District of Columbia,13.5,13.3,12.4
4,04000US13,Georgia,13.3,10.4,14.3


select all locations where the ratio of Senior Poverty to Poverty is greater than 0.95, meaning there are more people living in poverty than seniors living in poverty

In [254]:
%%bigquery
select Geo_ID, GeoName, (select (PovSenior/Poverty) as avg from Census_refined.Overall where Overall.Geo_ID = CensusLocation.Geo_ID) as SeniorToPovertyRatio
from Census_refined.CensusLocation
group by Geo_ID, GeoName
having SeniorToPovertyRatio > 0.90
order by SeniorToPovertyRatio DESC

,Geo_ID,GeoName,SeniorToPovertyRatio
0,04000US11,District of Columbia,0.985185
1,04000US25,Massachusetts,0.968085
2,04000US34,New Jersey,0.956522
3,04000US15,Hawaii,0.935484
4,04000US36,New York,0.923077
5,04000US72,Puerto Rico,0.917241


select all places that have an avg confirmed case more than the average confirmed cases in the US

In [248]:
%%bigquery
select l_id, Province_State, Country_Region, avg(Confirmed) as AvgConfirmed
from JHU_refined.USCovidCases
join JHU_refined.CovidLocation on CovidLocation.id = USCovidCases.l_id
group by l_id, Province_State, Country_Region
having avg(confirmed) >= (select avg(confirmed) from JHU_refined.USCovidCases where Country_Region = 'US' group by Country_Region)
order by l_id

,l_id,Province_State,Country_Region,AvgConfirmed
0,-7292501111997658044,Ohio,US,92713.431280
1,-6961371712520290149,California,US,456232.151659
2,-6821268968951623957,Maryland,US,83696.289100
3,-6184421288344879210,Illinois,US,195442.113744
4,-4727321918456285425,Massachusetts,US,111128.265403
5,-4647590228678710972,New Jersey,US,176337.729858
6,-4518569083713337097,Texas,US,408735.981043
7,-2243208293355838763,Louisiana,US,101980.374408
8,99786220189605989,North Carolina,US,118248.023697
9,595287789681633124,Georgia,US,176751.056872


join census table with covid table and show the state, mortality rate, percent of healworkers, percent of insurance 

In [276]:
%%bigquery
select Province_State, Mortality_Rate, HealthWorker, Insurance
from JHU_refined.USCovidCases
join JHU_refined.LocLinker
on l_id = Covid_ID
join Census_refined.Overall
on LocLinker.Geo_ID = Overall.Geo_ID
order by Last_Update DESC, Mortality_Rate DESC
limit 5

,Province_State,Mortality_Rate,HealthWorker,Insurance
0,New Jersey,6.452994,6.1,7.9
1,New York,6.368943,6.5,5.2
2,Connecticut,5.978880,6.6,5.9
3,Massachusetts,5.921271,6.5,3.0
4,New Hampshire,3.915759,6.7,6.3


join census table with covid table and show testing rate, percent of senior generation, percent of poverty

In [284]:
%%bigquery
select Province_State, Testing_Rate, SeniorGen, Poverty
from JHU_refined.USCovidCases
join JHU_refined.LocLinker
on l_id = Covid_ID
join Census_refined.Overall
on LocLinker.Geo_ID = Overall.Geo_ID
order by Last_Update DESC, Testing_Rate DESC
limit 5

,Province_State,Testing_Rate,SeniorGen,Poverty
0,North Dakota,118159.414851,15.8,10.6
1,Rhode Island,114967.041452,17.7,10.8
2,Alaska,106801.495465,12.4,10.1
3,Massachusetts,97504.868696,17.0,9.4
4,New York,80524.146710,16.9,13.0


CREATE DATA VISUALIZATION

In [273]:
%%bigquery
create view insights.v_High_Poverty as
select Overall.Geo_ID, CensusLocation.GeoName, Poverty, PovSenior, SeniorGen
from Census_refined.Overall
join Census_refined.CensusLocation on CensusLocation.Geo_ID = Overall.Geo_ID
where Poverty >= (select avg(Poverty) as avg_pov from Census_refined.Overall)
order by Geo_ID

""


In [286]:
%%bigquery
create or replace view insights.v_State_Testing_Rate_Correlation as
select Province_State, Testing_Rate, SeniorGen, Poverty
from JHU_refined.USCovidCases
join JHU_refined.LocLinker
on l_id = Covid_ID
join Census_refined.Overall
on LocLinker.Geo_ID = Overall.Geo_ID
order by Last_Update DESC, Testing_Rate DESC
limit 52 /*most recent data*/

""
